In [1]:
# neccessary imports

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.llms import OpenAI
from langchain_community.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

In [31]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [7]:
# creating vectorestore "Alice In Wonderland Story" - dont mind its my fav one 

vectorstore = FAISS.from_texts(
    ["Alice, a young girl, follows a rabbit down a rabbit hole and finds herself in a fantastical world.",
    "She encounters strange creatures, including the Cheshire Cat, the Mad Hatter, and the Queen of Hearts.",
    "The world she enters is nonsensical, with bizarre events and characters defying logic.",
    "Alice's size constantly changes, leading to amusing and challenging situations.",
     "The Cheshire Cat appears and disappears at will, offering cryptic advice.",
     "The Mad Hatter and the March Hare are stuck in perpetual tea time, engaging Alice in eccentric conversations.",
     "The Queen of Hearts is known for her love of croquet and her famous catchphrase, Off with their heads!",
     "Alice attends a trial where nonsensical accusations are made, and the Queen orders executions.",
     "Alice realizes the absurdity of Wonderland and challenges the Queen's authority.",
     "She eventually wakes up, realizing it was all a dream, leaving Wonderland's eccentric characters behind."], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

e:\AssistantChat\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [8]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [9]:
chain.invoke("who was alice?")

'Alice was a young girl who followed a rabbit down a rabbit hole and found herself in a fantastical world.'

In [10]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [12]:
chain.invoke({"question": "Why does Alice decide to leave Wonderland at the end of the story?", "language": "english"})

"Alice decides to leave Wonderland at the end of the story because she realizes the absurdity of Wonderland and challenges the Queen's authority. Additionally, she eventually wakes up, realizing it was all a dream, and leaves behind Wonderland's eccentric characters."

In [19]:
# creating app with qdrant - this time vectorestore ill be adding about ALbert Einsten, ask QnA about his work

from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings
import qdrant_client
import os

In [20]:
# create qdrant client

os.environ['QDRANT_HOST']
os.environ["QDRANT_API_KEY"]

client = qdrant_client.QdrantClient(
    os.getenv("QDRANT_HOST"),
    api_key=os.getenv("QDRANT_API_KEY")
)

In [21]:
# read the file
file="my.txt"
data=""

with open(file,'r') as f:
    data = f.read()

In [22]:
# split the text into chunks
#create a function to return chunks
def get_chunks(text):
    text_splitter=CharacterTextSplitter(
        separator= "\n",
        chunk_size=500,
        chunk_overlap=100, # second chunk start  character from 800, overlap is used to stop loosing chunk 
        length_function=len
    )

    chunks=text_splitter.split_text(text)
    return chunks

In [23]:
# get the chunks for the data
texts=get_chunks(data) 

In [24]:
len(texts)

7

In [25]:
len(data)

2530

In [26]:
# creating a new collection

vectors_config=models.VectorParams(
    # depends on model, we can google dimension. 1536 for openai
    # we are using openai embedding, for that size is 1536
    size=1536, 
    distance=models.Distance.COSINE)

client.create_collection(
    collection_name="Albert Einsten",
    vectors_config=vectors_config,
)

True

In [34]:
# create a vector store object using langchain

# if we want to use any other embedding, we need to change size

os.environ["OPENAI_API_KEY"]

embeddings = OpenAIEmbeddings()

vector_store = Qdrant(
    client=client, 
    collection_name="Albert Einsten", 
    embeddings=embeddings,
)

In [35]:
# add chunks to vector store
vector_store.add_texts(texts)

['fe8af872e7ce4c47a3dd445e391f6ad9',
 'a86b604f91b441b8872b4ffbc7ecadd0',
 '31f704b4baf54759874c88e4eccb616b',
 '77267bb5173c47be9ff0a320892b7767',
 'faa899f6706c44f1b3c047452c26bbfa',
 'bb174938e99b436e89d7659926e73490',
 '3186faed36984934b0393c2d2e841794']

In [36]:
retriever=vector_store.as_retriever()
print(retriever)

tags=['Qdrant', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x000001B1C3126910>


In [37]:
#plug vector store into retrieval chain

from langchain.chains import RetrievalQA
from langchain.schema import retriever
from langchain.llms import OpenAI

qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",  # Positional argument before keyword argument
    retriever=vector_store.as_retriever()
)

In [38]:
query = "What is Einsten theory?"
response = qa.run(query)

print(response)

 Einstein's theory is the general theory of relativity, which incorporates the force of gravity and provides a new understanding of how massive objects influence the fabric of spacetime. 
